In [80]:
import torch, math
torch.manual_seed(0)
dtype = torch.float64

m, n = 2, 3                     # outputs, inputs  (tiny toy layer)

# ----- tiny random tensors -----
W = torch.randn(m, n, requires_grad=True)   # The weight matrix W has rows that correspond one-to-one to the output neurons.
                                            # Row 0 are all the weights feeding neuron 0
x = torch.randn(n, requires_grad=True)      # input vector (n,)
g = torch.randn(m, requires_grad=True)      # gain vector
b = torch.randn(m, requires_grad=True)      # bias vector

# ----- forward pass -----
a = W @ x                                   # a_i is the pre-activation (the weighted sum before any non-linearity) of neuron i
                                            # The activation vector a collects those neurons: a_0, a_1, ..., a_{m-1}
                                            # So a_0 is literally the output of "neuron 0": a_0 = W0,q @ x_q
a.retain_grad()
u = torch.sqrt((a ** 2).mean()) # RMS scale (scalar)
v = g * (a / u) + b
v.retain_grad()
loss = v.pow(2).sum()           # simple scalar loss

# ----- Autograd reference -----
loss.backward()
grad_auto = W.grad.detach()   # (m × n)
bias_grad = b.grad.detach()
print("\nAutograd  ∂L/∂W\n", grad_auto, grad_auto.shape)


Autograd  ∂L/∂W
 tensor([[-0.0538, -0.1118,  0.0960],
        [ 0.3203,  0.6655, -0.5712]]) torch.Size([2, 3])


In [81]:
# ------------ analytic gradient ----------------------------
aL_av = 2 * v.detach()                                  # ∂L/∂v  (because L = Σ v_i²)
aL_av = aL_av.unsqueeze(0)
print("aL_av", aL_av, aL_av.shape)

aL_ab = d
print("∂L/∂b gradients match:", torch.allclose(aL_ab, b.grad))

aL_ag = aL_av * (a / u)
print("∂L/∂g gradients match:", torch.allclose(aL_ag, g.grad))

I = torch.eye(m) # (m, m)
a_aa = 1/u * (I - torch.outer(a, a) / (m * u**2)) # (m, m)
R = a_aa # (m, m)
R.retain_grad()

av_aa = torch.diag(g) @ R # (m, m)

# ∂a_i / ∂W_pq = δ_ip * x_q

# i is which activation we are taking the derivative of (e.g. a_0)
# p is the weight you perturb (i.e. which neuron the weight belongs to) -- row in W
# q is the column index of the weight (which input feature it multiplies) -- e.g. x_1

# If you perturb a weight in same row p, the change in activation i is "input * weight change"
# If you perturb a weight in different row p, the change in activation i is 0
aa_aW = torch.kron(I, x)
print("aa_aW", aa_aW, aa_aW.shape) # (m, m*n)

aL_aW = aL_av @ av_aa @ aa_aW
aL_aW = aL_aW.view(m, n)
print("∂L/∂W gradients match:", torch.allclose(aL_aW, W.grad))

aL_av tensor([[-0.7610, -1.9929]]) torch.Size([1, 2])
∂L/∂b gradients match: True
∂L/∂g gradients match: True
aa_aW tensor([[ 0.4033,  0.8380, -0.7193,  0.0000,  0.0000, -0.0000],
        [ 0.0000,  0.0000, -0.0000,  0.4033,  0.8380, -0.7193]],
       grad_fn=<UnsafeViewBackward0>) torch.Size([2, 6])
∂L/∂W gradients match: True


In [86]:
# column-major (Fortran / MATLAB default)
#“weights are stored column-by-column, so route the column-gradient (xᵀ) first, then scale each entire column by the inputs (Iₘ).
J = torch.kron(x.unsqueeze(0), I)

print("Partial derivative of Wx with respect to W (col-major)")
print("Kronecker Jacobian Iₘ ⊗ xᵀ:\n", J, "\n")

# Show the structure of the row-major Jacobian
print("Col-major structure:")
print("[[x1  0  x2   0   x3   0 ]")
print(" [ 0   x1   0  x2  0  x3]]\n")

# Row in W corresponds to a neuron
# Column in W corresponds to a weight that will multiply with feature in x (column in x)
# Result is the neuron's activation
# Ji,j = x_t * I_m = δ_{ip} * x_q

# Verify the Jacobian entries
for i, p, q in itertools.product(range(m), repeat=3):
    j = q*m + p
    if q < n:   # keep q in 0..n-1
        lhs = J[i, j].item()
        rhs = (1.0 if i==p else 0.0)*x[q].item()
        print(f"i={i}, p={p}, q={q}:  J = {lhs:.2f}  δ·x = {rhs:.2f}")


Partial derivative of Wx with respect to W (col-major)
Kronecker Jacobian Iₘ ⊗ xᵀ:
 tensor([[ 1.3000,  0.0000,  0.9000,  0.0000, -0.7000, -0.0000],
        [ 0.0000,  1.3000,  0.0000,  0.9000, -0.0000, -0.7000]]) 

Col-major structure:
[[x1  0  x2   0   x3   0 ]
 [ 0   x1   0  x2  0  x3]]

i=0, p=0, q=0:  J = 1.30  δ·x = 1.30
i=0, p=0, q=1:  J = 0.90  δ·x = 0.90
i=0, p=1, q=0:  J = 0.00  δ·x = 0.00
i=0, p=1, q=1:  J = 0.00  δ·x = 0.00
i=1, p=0, q=0:  J = 0.00  δ·x = 0.00
i=1, p=0, q=1:  J = 0.00  δ·x = 0.00
i=1, p=1, q=0:  J = 1.30  δ·x = 1.30
i=1, p=1, q=1:  J = 0.90  δ·x = 0.90


In [89]:
# row-major (C / PyTorch default)	
# weights are stored row-by-row, so route the row-gradient (Iₘ) first, then scale each entire row by the inputs (xᵀ)
J = torch.kron(I, x.unsqueeze(0))  # (m × m n)

print("Partial derivative of Wx with respect to W (row-major)")
print("Kronecker Jacobian Iₘ ⊗ xᵀ:\n", J, "\n")

# Show the structure of the row-major Jacobian
print("Row-major structure:")
print("[[x1  x2  x3   0   0   0 ]")
print(" [ 0   0   0  x1  x2  x3]]\n")

for i, p, q in itertools.product(range(m), repeat=3):
    j = p*n + q  # For row-major layout: j = p*n + q maps (p,q) to column index
                  # where p is output index (0..m-1), q is input index (0..n-1)
                  # This formula places each input x[q] in the correct column
                  # based on which output row p it contributes to
    if q < n:   # keep q in 0..n-1
        lhs = J[i, j].item()  # Left-hand side: actual Jacobian value
        rhs = (1.0 if i==p else 0.0)*x[q].item()  # Right-hand side: expected value (Kronecker delta * input)
        print(f"i={i}, p={p}, q={q}:  J = {lhs:.2f}  δ·x = {rhs:.2f}")


Partial derivative of Wx with respect to W (row-major)
Kronecker Jacobian Iₘ ⊗ xᵀ:
 tensor([[ 1.3000,  0.9000, -0.7000,  0.0000,  0.0000, -0.0000],
        [ 0.0000,  0.0000, -0.0000,  1.3000,  0.9000, -0.7000]]) 

Row-major structure:
[[x1  x2  x3   0   0   0 ]
 [ 0   0   0  x1  x2  x3]]

i=0, p=0, q=0:  J = 1.30  δ·x = 1.30
i=0, p=0, q=1:  J = 0.90  δ·x = 0.90
i=0, p=1, q=0:  J = 0.00  δ·x = 0.00
i=0, p=1, q=1:  J = 0.00  δ·x = 0.00
i=1, p=0, q=0:  J = 0.00  δ·x = 0.00
i=1, p=0, q=1:  J = 0.00  δ·x = 0.00
i=1, p=1, q=0:  J = 1.30  δ·x = 1.30
i=1, p=1, q=1:  J = 0.90  δ·x = 0.90
